In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 라이브러리 임포트 및 시드 고정

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import os
from sklearn.model_selection import StratifiedKFold

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED) # Seed 고정

# 데이터 불러오기

In [ ]:
!unzip /content/drive/MyDrive/250707_digital/open.zip

Archive:  /content/drive/MyDrive/250707_digital/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# Google Drive에 데이터가 있다면 경로를 지정합니다. (여기서는 /content 디렉토리에 업로드했다고 가정)
TRAIN_CSV = "/content/train.csv"
TEST_CSV = "/content/test.csv"
SUBMISSION_CSV = "/content/sample_submission.csv"

# 학습 데이터 불러오기
train_df = pd.read_csv(TRAIN_CSV, encoding="utf-8-sig")
test_df = pd.read_csv(TEST_CSV, encoding="utf-8-sig")
submission_df = pd.read_csv(SUBMISSION_CSV, encoding="utf-8-sig")
print("원본 학습 데이터 크기:", len(train_df))

원본 학습 데이터 크기: 97172


# TRAIN 데이터 전처리

In [ ]:
def minimal_preprocess(text):
    text = text.strip()
    text = re.sub(r'[\u4E00-\u9FFF]', '', text)                   # 한자 제거
    text = re.sub(r'<[^>]+>', '', text)                           # HTML 태그 제거
    text = re.sub(r'\(\s*[^\w가-힣]*\s*\)', '', text)             # 빈 괄호 제거
    text = re.sub(r'\([^\(\)]{0,20}[\?\~]{1,3}[^\(\)]{0,20}\)', '', text)  # ( ? ~ ? ) 제거
    text = re.sub(r'[.,]{3,}', '.', text)                         # ... → .
    text = re.sub(r'[()]{2,}', '', text)                          # 괄호 잔재 정리
    text = re.sub(r',\s*,+', ',', text)
    text = re.sub(r'\s+', ' ', text)                              # 중복 공백 제거
    return text

def split_into_paragraphs(text):
    # 문단 기준: 두 줄 개행 우선
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    # # 만약 너무 적게 쪼개졌으면 한 줄 개행으로 재시도
    if len(paragraphs) <= 1:
        paragraphs = [p.strip() for p in text.split('\n') if p.strip()]
    return paragraphs

def convert_train_to_paragraphs(train_df):
    rows = []
    for _, row in train_df.iterrows():
        title = row['title']
        full_text = row['full_text']  # ✅ 전처리 전 원본에서 문단 나눔
        label = row['generated']
        paragraphs = split_into_paragraphs(full_text)
        for idx, para in enumerate(paragraphs):
            cleaned_para = minimal_preprocess(para)  # ✅ 각 문단에 대해 전처리
            rows.append({
                'title': title,
                'paragraph_index': idx,
                'paragraph_text': cleaned_para,
                'generated': label
            })
    return pd.DataFrame(rows)

In [ ]:
paragraph_train = convert_train_to_paragraphs(train_df)

In [ ]:
# ✅ 1) paragraph_text가 NaN인 행 개수 확인
nan_cnt = paragraph_train['paragraph_text'].isna().sum()
print(f"paragraph_text NaN 개수: {nan_cnt}")

# ✅ 2) NaN 행 제거 및 인덱스 재정렬
paragraph_train = (
    paragraph_train
      .dropna(subset=['paragraph_text'])  # NaN 행 삭제
      .reset_index(drop=True)             # 인덱스 리셋
)

print("제거 후 데이터 크기:", len(paragraph_train))

paragraph_text NaN 개수: 0
제거 후 데이터 크기: 1226364


In [ ]:
paragraph_train = paragraph_train.rename(columns={'paragraph_text': 'full_text'})

In [ ]:
paragraph_train

,title,paragraph_index,full_text,generated
0,카호올라웨섬,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0
1,카호올라웨섬,1,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...,0
2,카호올라웨섬,2,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...,0
3,카호올라웨섬,3,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...,0
4,카호올라웨섬,4,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...,0
...,...,...,...,...
1226359,펩시 스터프,5,펩시 스터프 프로모션 이후에 펩시와 코카-콜라 모두 몇 년 동안에 걸쳐 원래의 캠페...,0
1226360,펩시 스터프,6,코카-콜라 컴퍼니는 2005년에 소비자들이 캐나다에서 패키지에 인쇄된 포인트를 수집...,0
1226361,펩시 스터프,7,펩시코는 2008년 2월 1일에 아마존 MP3(나중에 아마존 뮤직으로 이름을 바꿈)...,0
1226362,펩시 스터프,8,2015년에 펩시 패스로 재출시된 이 프로그램은 다양한 방법으로 소비자가 포인트를 ...,0


In [ ]:
# 1) 문단 길이(문자 수) 계산
paragraph_train['char_len'] = paragraph_train['full_text'].str.len()

# 2) 라벨(generated)별 35 % · 95 % 퍼센타일 계산
percentiles = (
    paragraph_train
      .groupby('generated')['char_len']
      .quantile([0.35, 0.95])        # 두 지점 한 번에 구함
      .unstack(level=1)              # 보기 편하게: index=라벨, columns=p35/p95
      .rename(columns={0.35: 'p35', 0.95: 'p95'})
)

print("라벨별 문단 길이 퍼센타일")
print(percentiles)

라벨별 문단 길이 퍼센타일
             p35    p95
generated              
0          102.0  448.0
1          109.0  432.0


In [ ]:
# 3) 위 기준을 이용해 필터링
mask = paragraph_train.apply(
    lambda r: percentiles.loc[r['generated'], 'p35'] <= r['char_len'] <= percentiles.loc[r['generated'], 'p95'],
    axis=1
)

filtered_df = (
    paragraph_train[mask]
      .reset_index(drop=True)
      .drop(columns=['char_len'])   # 길이 컬럼이 필요 없으면 제거
)

print(f"필터링 전: {len(paragraph_train)}  →  필터링 후: {len(filtered_df)}")

필터링 전: 1226364  →  필터링 후: 737516


In [ ]:
# ✅ 라벨별 개수 확인
label_counts = filtered_df['generated'].value_counts()
print("라벨별 개수 (필터링 후):")
print(label_counts)

# ✅ 1:1 언더샘플링 ─ 소수 클래스(1번)의 개수만큼만 0번에서 랜덤 추출
min_cnt = label_counts.min()            # 1번 라벨 개수
balanced_df = (
    filtered_df
      .groupby('generated', group_keys=False)
      .apply(lambda x: x.sample(n=min_cnt, random_state=SEED))  # 동일 개수 샘플링
      .reset_index(drop=True)
)

print("\n언더샘플링 후 라벨별 개수:")
print(balanced_df['generated'].value_counts())

# balanced_df 가 1:1 비율로 균형 잡힌 학습용 데이터입니다.

라벨별 개수 (필터링 후):
generated
0    676754
1     60762
Name: count, dtype: int64

언더샘플링 후 라벨별 개수:
generated
0    60762
1    60762
Name: count, dtype: int64


/tmp/ipython-input-13-2796369302.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_cnt, random_state=SEED))  # 동일 개수 샘플링


In [ ]:
N_SPLITS   = 4
OUTPUT_DIR = "/content/kfold_csv"   # 원하는 경로로 변경 가능

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ──────────────────────────────
# Stratified 4-Fold 분할
# ──────────────────────────────
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

In [ ]:
# fold별 DataFrame을 담아 둘 컨테이너
fold_dfs = {}          # {0: fold0_df, 1: fold1_df, ...}

for fold, (_, val_idx) in enumerate(skf.split(balanced_df, balanced_df['generated'])):
    # fold별 검증 세트만 따로 추출
    fold_df = (
        balanced_df.iloc[val_idx]          # fold 인덱스 추출
          .sample(frac=1, random_state=SEED)  # ★ 행 순서 셔플
          .reset_index(drop=True)
    )
    fold_dfs[fold] = fold_df           # 저장

    # 라벨 비율(1:1 여부) 확인
    counts = fold_df['generated'].value_counts().sort_index()
    print(f"Fold {fold}  →  0:{counts[0]} | 1:{counts[1]}   (총 {len(fold_df)}개)")

# 이제 fold_dfs[0] ~ fold_dfs[4] 에서 각 fold의 데이터프레임을 바로 사용할 수 있습니다.
# (저장하려면 이후에 to_csv 호출만 추가하면 됩니다)

Fold 0  →  0:15191 | 1:15190   (총 30381개)
Fold 1  →  0:15191 | 1:15190   (총 30381개)
Fold 2  →  0:15190 | 1:15191   (총 30381개)
Fold 3  →  0:15190 | 1:15191   (총 30381개)


In [ ]:
fold_dfs[0]

,title,paragraph_index,full_text,generated
0,카티푸난,1,필리핀의 민족주의 단체 카티푸난은 1892년 스페인으로부터 독립을 꿈꾸며 안드레스 ...,1
1,박흥용,1,"박흥용은 중학교 2학년때부터 만화가를 장래희망으로 정했으며, 1975년부터 만화를 ...",0
2,온라인 저널리즘,6,크레이그(Craig)처럼 온라인을 통해 정치 경제 사회 문화 시사 등 다양한 분야의...,1
3,미국-중국 무역 전쟁,34,"2018년 9월, 기업 연합은 제안된 관세에 항의하기 위해 ""관세가 심장부를 해친다...",1
4,강간,55,"2013년 6월 19일, 강간죄와 강제추행죄 등을 고소가 있어야 공소를 제기할 수 ...",0
...,...,...,...,...
30376,창원 유씨,4,문헌에 따르면 고구려는 건국 초기인 1세기 무렵부터 성씨를 사용하기 시작했고 백제는...,1
30377,실스 임 엥가딘/실,22,철학자 프리드리히 니체는 1881년과 1883년과 1888년 사이에 실스에서 여름을...,0
30378,채상덕,1,1920년 일본군의 만주 출병 이후 남북 만주 각지에 분산된 독립군들이 이념적으로도...,0
30379,사포서,1,조선 초기에 왕실 채소를 담당하는 침장고가 설치되었습니다. 처음 설치될 당시 제거나...,1


In [ ]:
# ─────────────────────────────────────────
# ❶ fold_dfs 사전에 ID 컬럼 추가하기
#    형식: FOLD{fold 번호}_{5자리 순번}
# ─────────────────────────────────────────
for fold, df in fold_dfs.items():
    df.insert(
        0,                                   # 맨 앞 컬럼으로 삽입
        'id',
        [f"FOLD{fold}_{i:05d}" for i in range(len(df))]
    )
    fold_dfs[fold] = df                      # 덮어쓰기(선택)

# 확인용 출력
for fold in range(len(fold_dfs)):
    print(f"▶ Fold {fold}  첫 3개 ID:")
    print(fold_dfs[fold][['id', 'generated']].head(3), "\n")

▶ Fold 0  첫 3개 ID:
            id  generated
0  FOLD0_00000          1
1  FOLD0_00001          0
2  FOLD0_00002          1 

▶ Fold 1  첫 3개 ID:
            id  generated
0  FOLD1_00000          1
1  FOLD1_00001          0
2  FOLD1_00002          1 

▶ Fold 2  첫 3개 ID:
            id  generated
0  FOLD2_00000          1
1  FOLD2_00001          0
2  FOLD2_00002          1 

▶ Fold 3  첫 3개 ID:
            id  generated
0  FOLD3_00000          1
1  FOLD3_00001          0
2  FOLD3_00002          1 



In [ ]:
# 이미 셔플·ID 부여가 끝난 fold_dfs 저장
for fold, df in fold_dfs.items():
    save_path = os.path.join(OUTPUT_DIR, f"fold{fold}.csv")
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✓ fold{fold}.csv  →  {save_path}  (행 {len(df)})")

✓ fold0.csv  →  /content/kfold_csv/fold0.csv  (행 30381)
✓ fold1.csv  →  /content/kfold_csv/fold1.csv  (행 30381)
✓ fold2.csv  →  /content/kfold_csv/fold2.csv  (행 30381)
✓ fold3.csv  →  /content/kfold_csv/fold3.csv  (행 30381)


# TEST 데이터 전처리

In [ ]:
test_df

,ID,title,paragraph_index,paragraph_text
0,TEST_0000,공중 도덕의 의의와 필요성,0,도덕이란 원래 개인의 자각에서 출발해 자기 의지로써 행동하는 일이다. 그러므로 도덕...
1,TEST_0001,공중 도덕의 의의와 필요성,1,도덕은 단순히 개인의 문제나 사회의 문제로 한정될 수 없다. 개인적인 측면과 사회적...
2,TEST_0002,공중 도덕의 의의와 필요성,2,"여기에 이른바 공중도덕은 실천적, 사회적 도덕의 한 부문이다. 즉, 공중 도덕이라 ..."
3,TEST_0003,공중 도덕의 의의와 필요성,3,우리가 공동 생활을 하는 데 있어서 공중 도덕이 필요함은 위에서 말한 것처럼 알 수...
4,TEST_0004,풍습과 그 개선,0,인간 사회에서는 다 함께 지켜야 할 어떤 기준이 있어 이를 따르면 옳다고 하고 따르...
...,...,...,...,...
1957,TEST_1957,저작권! 내가 먼저 지켜야지,11,"인터넷에는 음악뿐만 아니라 인터넷소설, 영화, 애니메이션 등 내가 좋아하는 것들이 ..."
1958,TEST_1958,저작권! 내가 먼저 지켜야지,12,하지만 이 경험을 통해 나는 달라진 시각을 갖게 되었다. 이제는 내가 좋아하는 콘텐...
1959,TEST_1959,저작권! 내가 먼저 지켜야지,13,그런데 누군가는 아무 노력이나 허락 없이 사용한다면 불공평하다. 우리나라는 인기 있...
1960,TEST_1960,저작권! 내가 먼저 지켜야지,14,"우리들이 우리나라의 노래, 영화, 드라마, 애니메이션 등의 저작권을 지켜 주어야 다..."


In [ ]:
# paragraph_text에 전처리 적용
test_df['paragraph_text'] = test_df['paragraph_text'].apply(minimal_preprocess)

In [ ]:
# 저장
test_df.to_csv("/content/kfold_csv/test_preprocessed.csv", index=False, encoding='utf-8-sig')

In [ ]:
# sample_submission.csv 를 kfold_csv 폴더로 복사
!cp /content/sample_submission.csv /content/kfold_csv/